In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

yu.flag_fast=True

In [ ]:
ensembles=['cA211.530.24','cA2.09.48']

ens2data={}
for ens in ensembles:
    path={
        'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_c/data/cA211.530.24_NST_a_daint.h5',
        'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_c/data/cA2.09.48_NST_c_booster.h5',
        # 'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_c/cA2.09.48/data_merge/NST_c.h5',
        }[ens]
    ens2data[ens]=yu.load(path)
    print('Ncfg='+str(yu.deepKey(ens2data[ens]['2pt'],2).shape[0]))
    print()

lat=None
data=None
Ncfg=None
Tpack=None
    
def updateEns(ens):
    global lat,data,Ncfg,Tpack
    lat=yu.LatticeEnsemble(ens)
    data=ens2data[ens]
    Ncfg=yu.deepKey(data['2pt'],2).shape[0]
    Tpack=yu.TPack[ens]
updateEns(ensembles[0])

In [ ]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    t=data['2pt'][opa+'_'+opb] if opa+'_'+opb in data['2pt'] else {}
    res=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if yu.getNpars(opa,opb)==(2,1):
        res+=np.conj(get2pt(opb,opa,set(diags)-{'N-pi0i',}))
    if opb+'_'+opa in data['2pt'].keys(): # contibution for c.c. of T-pi0f
        t=data['2pt'][opb+'_'+opa]; diag='T-pi0f'
        res+= np.conj(t[diag]) if diag in t.keys() and diag in diags else 0
    return res
def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    t=data['3pt'][opa+'_'+opb][insert] if opa+'_'+opb in data['3pt'] else {}
    res=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if yu.getNpars(opa,opb)==(2,1):
        res+=np.conj(get3pt(opb,opa,insert,set(diags)-{'N-jPi','N-j-pi0i'})[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    if opb+'_'+opa in data['3pt'].keys(): # contibution for c.c. of T-pi0f-j, T-jPf, B3pt-pi0f, W3pt-pi0f, Z3pt-pi0f
        t=data['3pt'][opb+'_'+opa]; diags_cc=['T-pi0f-j','T-jPf','B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f']
        res+= np.conj(np.sum([t[diag][:,::-1] if diag in t.keys() and diag in diags else np.zeros([Ncfg,int(insert.split('_')[-1])+1]) for diag in diags_cc],axis=0))*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all,NpiNpi=False):
    if flags['ll3pt']:
        flags['ll3pt']=False
        opas_flip=[yu.op_flipl(op) for op in opas]; opbs_flip=[yu.op_flipl(op) for op in opbs]
        sgns=np.array([[yu.op_getl_sgn(opa)*yu.op_getl_sgn(opb) for opb in opbs] for opa in opas])
        sgns*=yu.fourCPTstar[insert.split('_')[0]]
        t=(get3ptMat(opas,opbs,insert,diags=diags,NpiNpi=NpiNpi)+np.conj(get3ptMat(opas_flip,opbs_flip,insert,diags=diags,NpiNpi=NpiNpi))*sgns[None,None,:,:])/2
        flags['ll3pt']=True
        return t
    skipNpiNpi=lambda opa,opb: yu.getNpars(opa,opb)==(2,2) and not NpiNpi
    t=np.transpose([[get3pt(opa,opb,insert,diags) if not skipNpiNpi(opa,opb) else np.zeros([Ncfg,int(insert.split('_')[-1])+1])  for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags,NpiNpi)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

In [ ]:
def run2pt(pt,ofa,ofb,prefactor='re',diags=yu.diags_all,title=None,ylim1=None,ylim2=None):
    fig, axs = yu.getFigAxs(1,2)
    title=title if title=='None' else 'pt={}; ofa={}; ofb={}; prefactor={}'.format(pt,ofa,ofb,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=1; tmax=15
    
    for l,color,xshift in zip(['l1','l2'],['r','b'],[-0.1,0.1]):
        opa,opb=yu.getopab(pt,l,ofa,ofb)
        dat=get2pt(opa,opb,diags)
        dat=dat[:300]
        
        irow=0; icol=0
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            t=np.log(t/np.roll(t,-1,axis=0))
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim2)
        
updateEns(ensembles[1])
        
# run2pt('0,0,0',('a','p'),('a','p'))
# run2pt('0,0,1',('a','p'),('a','p'))

# run2pt('0,0,0',('a','p'),('N1pi1,a','n,pi+'))
# run2pt('0,0,0',('a','p'),('N1pi1,a','p,pi0'),diags=['T'])
# run2pt('0,0,0',('a','p'),('N1pi1,a','p,pi0'),diags=['N-pi0i'])

# run2pt('0,0,1',('a','p'),('N0pi1,a','n,pi+'))
# run2pt('0,0,1',('a','p'),('N0pi1,a','n,pi+'),prefactor='im')
# run2pt('0,0,1',('a','p'),('N0pi1,a','p,pi0'))
# run2pt('0,0,1',('a','p'),('N0pi1,a','p,pi0'),prefactor='im')

# run2pt('0,0,1',('a','p'),('N1pi0,a','n,pi+'))
# run2pt('0,0,1',('a','p'),('N1pi0,a','n,pi+'),prefactor='im')

# run2pt('0,0,1',('a','p'),('N1pi0,a','p,pi0'),diags=['N-pi0i'],ylim1=[-5e-9,5e-9])
# run2pt('0,0,1',('N1pi0,a','p,pi0'),('a','p'),diags=['N-pi0f'],ylim1=[-5e-9,5e-9])
# run2pt('0,0,1',('a','p'),('N1pi0,a','p,pi0'),diags=['N-pi0i'],prefactor='im')
# run2pt('0,0,1',('N1pi0,a','p,pi0'),('a','p'),diags=['N-pi0f'],prefactor='im')


# run2pt('0,0,0',('N1pi1,a','n,pi+'),('N1pi1,a','p,pi0'),diags={'B2pt',},ylim2=[0.,0.8])
# run2pt('0,0,0',('N1pi1,a','n,pi+'),('N1pi1,a','p,pi0'),diags={'Z2pt',},ylim2=[0.,0.8])
# run2pt('0,0,0',('N1pi1,a','p,pi0'),('N1pi1,a','n,pi+'))
# run2pt('0,0,0',('N1pi1,a','p,pi0'),('N1pi1,a','p,pi0'))

# run2pt('0,0,1',('N1pi0,a','n,pi+'),('N0pi1,a','n,pi+'),diags={'B2pt',},ylim2=[0.,1])
# run2pt('0,0,1',('N1pi0,a','n,pi+'),('N0pi1,a','n,pi+'),diags={'Z2pt',},ylim2=[0.,1])

# run2pt('0,0,0',('N1pi1,a','n,pi+'),('N1pi1,a','n,pi+'),diags={'N-P'})
# run2pt('0,0,0',('N1pi1,a','n,pi+'),('N1pi1,a','n,pi+'),diags={'N-P','B2pt'})

In [ ]:
def run2pt_Npi0(pt,ofa,ofb,prefactor='re',title=None,ylim1=None,ylim2=None):
    fig, axs = yu.getFigAxs(1,3)
    title=title if title=='None' else 'pt={}; ofa={}; ofb={}; prefactor={}'.format(pt,ofa,ofb,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=0; tmax=15
    
    for l,color,xshift in zip(['l1','l2'],['r','b'],[-0.1,0.1]):
        opa,opb=yu.getopab(pt,l,ofa,ofb)
        
        irow=0; icol=0
        dat=get2pt(opa,opb,['N-pi0i'])
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        dat=get2pt(opb,opa,['N-pi0f'])
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=2
        dat=(get2pt(opa,opb,['N-pi0i'])+get2pt(opb,opa,['N-pi0f']))/2
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(np.array(ylim1)/5)
        axs[irow,icol].set_ylim(ylim2)

# run2pt_Npi0('0,0,0',('a','p'),('N1pi1,a','p,pi0'),ylim1=[-3e-9,1e-9],ylim2=[-3e-9,1e-9])
# run2pt_Npi0('0,0,1',('a','p'),('N0pi1,a','p,pi0'),ylim1=[-3e-9,1e-9],ylim2=[-3e-9,1e-9])
# run2pt_Npi0('0,0,1',('a','p'),('N1pi0,a','p,pi0'),ylim1=[-10e-9,10e-9],ylim2=[-1e-9,1e-9])

In [ ]:
def run2pt_cc(pt,of1,of2,prefactor='re',diags=yu.diags_all,title=None,ylim=None):
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'pt={}; of1={}; of2={}, prefactor={}'.format(pt,of1,of2,prefactor)
    # fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=0; tmax=15
    
    for ofs,isoFactor,color,xshift in zip([(of1,of2),(of2,of1)],[1,1 if prefactor==1 else -1],['r','b'],[-0.1,0.1]):
        opa,opb=yu.getopab(pt,'l1',ofs[0],ofs[1])
        
        irow=0; icol=0
        dat=get2pt(opa,opb,diags)
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)

ri='re'
# run2pt_cc('0,0,0',('a','p'),('N1pi1,a','p,pi0'),prefactor=ri,diags=['N-pi0i','N-pi0f'])
# run2pt_cc('0,0,1',('a','p'),('N1pi0,a','p,pi0'),prefactor=ri,diags=['N-pi0i','N-pi0f'])
# run2pt_cc('0,0,1',('a','p'),('N0pi1,a','p,pi0'),prefactor=ri,diags=['N-pi0i','N-pi0f'])
# run2pt_cc('0,0,1',('N1pi0,a','n,pi+'),('N1pi0,a','p,pi0'),prefactor=ri,diags=['W2pt'])
# run2pt_cc('0,0,1',('N1pi0,a','p,pi0'),('N0pi1,a','n,pi+'),prefactor=ri,diags=yu.diags_all)

In [ ]:
def run2pt_iso_NNpi(pt,occ,prefactor='re',title=None,ylim=None):
    fig, axs = yu.getFigAxs(1,2)
    title=title if title=='None' else 'pt={}; occ={}; prefactor={}'.format(pt,occ,prefactor)
    fig.suptitle(title)
    yu.addColHeader(axs,['loopless','all'])
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=1; tmax=15
    
    for fla,isoFactor,color,xshift in zip(['n,pi+','p,pi0'],[1,(-1*np.sqrt(2))],['r','b'],[-0.1,0.1]):
        opa,opb=yu.getopab(pt,'l1',('a','p'),(occ,fla))
        
        irow=0; icol=0
        dat=get2pt(opa,opb,yu.diags_loopless)
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)
        
        irow=0; icol=1
        dat=get2pt(opa,opb,yu.diags_all)
        if fla=='p,pi0':
            dat=get2pt(opa,opb,['T'])+get2pt(opb,opa,['N-pi0f'])
            dat=get2pt(opa,opb,['T'])+(get2pt(opb,opa,['N-pi0f'])+np.conj(get2pt(opa,opb,['N-pi0i'])))/2
            # dat=get2pt(opa,opb,['T'])+get2pt(opa,opb,['N-pi0i'])
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)

# ri='re'
# ylim=[-1e-9,1e-9]; ylim=None
# run2pt_iso_NNpi('0,0,0','N1pi1,a',prefactor=ri,ylim=ylim)
# run2pt_iso_NNpi('0,0,1','N1pi0,a',prefactor=ri,ylim=ylim)
# run2pt_iso_NNpi('0,0,1','N0pi1,a',prefactor=ri,ylim=ylim)

In [ ]:
def run2pt_iso_NpiNpi(pt,occ,prefactor='re',title=None,ylim=None):
    fig, axs = yu.getFigAxs(1,2)
    title=title if title=='None' else 'pt={}; occ={}; prefactor={}'.format(pt,occ,prefactor)
    fig.suptitle(title)
    yu.addColHeader(axs,['loopless','all'])
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    tmin=1; tmax=15
    for flas,isoFactor,color,xshift in zip([('n,pi+','n,pi+'),('p,pi0','n,pi+'),('p,pi0','p,pi0')],[1,-1*np.sqrt(2),2],['r','g','b'],[-0.1,0,0.1]):
    # for flas,isoFactor,color,xshift in zip([('n,pi+','n,pi+'),('n,pi+','p,pi0'),('p,pi0','p,pi0')],[1,-1*np.sqrt(2),2],['r','g','b'],[-0.1,0,0.1]):
        opa,opb=yu.getopab(pt,'l1',(occ,flas[0]),(occ,flas[1]))
        
        irow=0; icol=0
        dat=get2pt(opa,opb,yu.diags_loopless)
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)
        
        irow=0; icol=1
        dat=get2pt(opa,opb,yu.diags_all)
        
        def func(dat):
            t=np.mean(dat,axis=0)
            t=np.real(prefactor*isoFactor*t)
            return t
        mean,err,cov=yu.jackknife(dat,func)
        ind=0
        tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        axs[irow,icol].set_ylim(ylim)
        
ri='re'
# run2pt_iso_NpiNpi('0,0,0','N1pi1,a',prefactor=ri,ylim=[0,1e-10])
# run2pt_iso_NpiNpi('0,0,1','N1pi0,a',prefactor=ri,ylim=[0,1e-10])
# run2pt_iso_NpiNpi('0,0,1','N0pi1,a',prefactor=ri,ylim=[0,1e-10])

In [ ]:
def run2ptGEVP(pt,ofs,diags=yu.diags_all,title=None,ylim1=None,ylim2=None,compQ=False):
    fig, axs = yu.getFigAxs(1,2) if compQ==False else yu.getFigAxs(1,3)
    title=title if title=='None' else 'pt={}; ofs={}'.format(pt,ofs)
    fig.suptitle(title)
    
    Nop=len(ofs)
    tRef=1; tmax0=15
    tmin= 1-tRef if tRef<0 else tRef+1
    
    for l,color,xshift in zip(['l1','l2'],['r','b'],[-0.1,0.1]):
        ops=yu.getops(pt,l,ofs)
        dat=get2ptMat(ops,diags)
        print(dat.shape)
        t=dat[:,1,1,1]
        for i,cfg in enumerate(cfgs):
            if np.abs(t[i])>1e-5 or np.abs(t[i])<1e-6:
                print(cfg, t[i])
            
        
        irow=0; icol=0
        def func(dat):
            t=np.mean(dat,axis=0)
            print(t.shape)
            print(t[1])
            print(t[10])
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0)) if tRef>=0 else np.log(eVals)/tRef
            return En.T
        func(dat)
        1/0
        mean,err,cov=yu.jackknife(dat,func)
        for ind in range(Nop):
            tmax=tmax0-3*ind
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','d','o'][ind])
            axs[irow,icol].set_xticks(range(0,tmax0,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        for j in range(2):
            def func(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,j,:].T
                t=t/t[0,:]
                return np.abs(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(Nop):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                # print(tMean)
                # print(tErr)
            
                axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','<','>'][ind])
                axs[irow,icol].set_xticks(range(0,tmax0,2))  
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_ylim(ylim2)
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])
        
        # if compQ:
        #     irow=0; icol=2
        #     def func(dat):
        #         t=np.mean(dat,axis=0)
        #         (eVals,eVecs,comp)=yu.GEVP(t,tRef,compQ)
        #         t=np.abs(comp/(comp[:,range(Nop),range(Nop)])[:,:,None])
        #         return [t[:,0,-1],t[:,-1,0]]
        #     mean,err,cov=yu.jackknife(dat,func)
        #     for ind in range(2):
        #         tmax=tmax0
        #         tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            
        #         axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','<','>'][ind])
        #         axs[irow,icol].set_xticks(range(0,tmax0,2))  
        #     axs[irow,icol].set_ylim([0,1])
            # t=np.array([mean[0],err[0]])
            # print(t.T)
            # t=np.array([mean[1],err[1]])
            # print(t.T)
            # print()
            
# updateEns(ensembles[1])
# run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','n,pi+')],ylim1=[0.3,1.1],diags={'N','T','N-P','B2pt'})  # t0~5
            
# run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','n,pi+'),('N1pi1,a','p,pi0')],ylim1=[0.3,1.1])        
# run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','12')],ylim1=[0.3,1.1],compQ=True)  # t0~5
# run2ptGEVP('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],ylim1=[0.3,1.1],compQ=True)   # t0~3

In [ ]:
def run2ptGEVP_get_xy(pt,ofs,diags=yu.diags_all,title=None,ylim1=None,ylim2=None):
    fig, axs = yu.getFigAxs(1,2)
    title=title if title=='None' else 'pt={}; ofs={}'.format(pt,ofs)
    fig.suptitle(title)
    
    Nop=len(ofs)
    tRef=5; tmax0=15
    tmin= 1-tRef if tRef<0 else tRef+1
    
    for l,color,xshift in zip(['l1','l2'],['r','b'],[-0.1,0.1]):
        ops=yu.getops(pt,l,ofs)
        dat=get2ptMat(ops,diags)
        
        irow=0; icol=0
        def func(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0)) if tRef>=0 else np.log(eVals)/tRef
            return En.T
        mean,err,cov=yu.jackknife(dat,func)
        for ind in range(Nop):
            tmax=tmax0-3*ind
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','d','o'][ind])
            axs[irow,icol].set_xticks(range(0,tmax0,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        def func(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            t=np.real([eVecs[:,i,0]*eVecs[:,0,i]/eVecs[:,0,0]/eVecs[:,i,i] for i in range(1,Nop)])
            return t
        mean,err,cov=yu.jackknife(dat,func)
        for ind in range(Nop-1):
            tmax=tmax0
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]

            axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['d','o'][ind])
            axs[irow,icol].set_xticks(range(0,tmax0,2))  
            
            # axs[irow,icol].set_yscale('log')
            # axs[irow,icol].set_ylim(ylim2)
            # axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])
            
# run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','n,pi+'),('N1pi1,a','p,pi0')],ylim1=[0.3,1.1])        
# run2ptGEVP_get_xy('0,0,0',[('a','p'),('N1pi1,a','12')],ylim1=[0.3,1.1])  # t0~5
# run2ptGEVP_get_xy('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],ylim1=[0.3,1.1])   # t0~3

In [ ]:
def run2ptGEVP_test_t0(pt,ofs,state_n,op_i,diags=yu.diags_all,title=None,ylim1=None,ylim2=None,compQ=False):
    fig, axs = yu.getFigAxs(1,2) if compQ==False else yu.getFigAxs(1,3)
    title=title if title=='None' else 'pt={}; ofs={}'.format(pt,ofs)
    fig.suptitle(title)
    
    Nop=len(ofs)
    
    l='l1'
    for t0,color,xshift in zip([1,2,4,6],['r','g','b','cyan'],[-0.1,0,0.1,0.2]):
        tRef=t0; tmax0=15
        tmin= 1-tRef if tRef<0 else tRef+1
        
        ops=yu.getops(pt,l,ofs)
        dat=get2ptMat(ops,diags)
        
        irow=0; icol=0
        def func(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0)) if tRef>=0 else np.log(eVals)/tRef
            return En.T
        mean,err,cov=yu.jackknife(dat,func)
        for ind in [state_n]:
            tmax=tmax0
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt='s')
            axs[irow,icol].set_xticks(range(0,tmax0,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        for j in range(1):
            def func(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,j,:].T
                t=t/t[0,:]
                return np.abs(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in [op_i]:
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            
                axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt='s')
                axs[irow,icol].set_xticks(range(0,tmax0,2))  
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3] if ylim2==None else ylim2)
                
        
        # if compQ:
        #     irow=0; icol=2
        #     def func(dat):
        #         t=np.mean(dat,axis=0)
        #         (eVals,eVecs,comp)=yu.GEVP(t,tRef,compQ)
        #         t=np.abs(comp/(comp[:,range(Nop),range(Nop)])[:,:,None])
        #         return [t[:,0,-1],t[:,-1,0]]
        #     mean,err,cov=yu.jackknife(dat,func)
        #     for ind in range(2):
        #         tmax=tmax0
        #         tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            
        #         axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','<','>'][ind])
        #         axs[irow,icol].set_xticks(range(0,tmax0,2))  
        #     axs[irow,icol].set_ylim([0,1])
            # t=np.array([mean[0],err[0]])
            # print(t.T)
            # t=np.array([mean[1],err[1]])
            # print(t.T)
            # print()

# run2ptGEVP_test_t0('0,0,0',[('a','p'),('N1pi1,a','12')],0,1,ylim1=[0.53,0.58],ylim2=[1e-1,0.5]) 
# # run2ptGEVP_test_t0('0,0,1',[('a','p'),('N1pi0,a','12')],0,1,ylim1=[0.6,0.62],ylim2=[1e-2,3e-2]) 
# # run2ptGEVP_test_t0('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],0,1,ylim1=[0.6,0.62],ylim2=[1e-2,3e-2]) 
# run2ptGEVP_test_t0('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],0,2,ylim1=[0.6,0.62],ylim2=[5e-2,1e-1]) 

In [ ]:
tfs=[10,12,14]

def run3pt(opa,opb,insert,prefactor,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    
    needsVEV=yu.needsVEV(opa,opb,insert) and 'N-j' in diags
    # needsVEV=False
    
    dat3pt=[get3pt(opa,opb,insert+'_'+str(tf),diags)[:,1:tf] for tf in tfs]
    datVEVj=[data['VEV']['j'][insert]] if needsVEV else []
    datVEV2pt=[get2pt(opa,opb,yu.diags_all)] if needsVEV else []
    dat=[dat3pt,datVEV2pt,datVEVj]

    def func(dat):
        t=yu.meanDeep(dat)
        t=[(t[0][i]-t[2][0]*t[1][0][tf]) for i,tf in enumerate(tfs)] if needsVEV else [t[0][i] for i,tf in enumerate(tfs)] 
        t=yu.prefactorDeep(t,prefactor)
        return t

    (mean,err,cov)=yu.jackknife(dat,func)

    irow=0; icol=0
    for i_tf,tf in enumerate(tfs):
        tMean=mean[i_tf];tErr=err[i_tf]
        # print(tMean)
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
    axs[irow,icol].set_xticks(range(-6,7,1))
    axs[irow,icol].set_ylim(ylim1)
    
# run3pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'id_j+',1,diags={'N-j'})
# run3pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'g5_j-',1,diags={'N-j'})

# run3pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'gt_j+',1,diags={'NJN'})
# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'gt_j+',1,diags={'NJN'})

# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('N0pi1,a','n,pi+')),'gt_j+',1,diags=yu.diags_all)

# run3pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('N0pi1,a','p,pi0')),'g5gt_j-',1,diags=['N-j-pi0i'])
# run3pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('N0pi1,a','p,pi0')),'g5gt_j-',1,diags=['N-jPi'])

# run3pt(yu.getop('0,0,0','l1',('N1pi1,a','p,pi0')),yu.getop('0,0,0','l1',('N1pi1,a','p,pi0')),'gt_j+',1,diags=yu.diags_all)

# run3pt(yu.getop('0,0,1','l1',('N0pi1,a','p,pi0')),yu.getop('0,0,1','l2',('N0pi1,a','p,pi0')),'g5gx_j-',-1j,diags=yu.diags_all)
# run3pt(yu.getop('0,0,1','l1',('N0pi1,a','p,pi0')),yu.getop('0,0,1','l2',('N0pi1,a','p,pi0')),'g5gx_j-',-1j,diags={
#                                                                                                                   'NJN-P', 'NJN-pi0f-pi0i',
#                                                                                                                 #   'B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f',
#                                                                                                                 #   'B2pt-j','W2pt-j','Z2pt-j',
#                                                                                                                 #   'N-pi0f-j-pi0i',
#                                                                                                                 #   'N-pi0f-jPi','N-jPf-pi0i',
#                                                                                                                 #   'N-P-j',
#                                                                                                                 #   'T-jPf','T-pi0f-j'
#                                                                                                                   })

# run3pt(yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),'g5gz_j-',-1j,diags=yu.diags_all)
# run3pt(yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),'g5gz_j-',-1j,diags={
#                                                                                                                     'NJN-pi0f-pi0i',
#                                                                                                                   'NJN-P',
#                                                                                                                #   'N-pi0f-j-pi0i',
#                                                                                                                 #   'B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f',
#                                                                                                                 #   'N-pi0f-jPi','N-jPf-pi0i',
#                                                                                                                 #   'B2pt-j','W2pt-j','Z2pt-j',
#                                                                                                                 #   'N-pi0f-j-pi0i',
#                                                                                                                 #   'N-P-j',
#                                                                                                                 #   'T-jPf','T-pi0f-j'
#                                                                                                                   })

# updateEns(ensembles[1])
# run3pt(yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),'g5_j-',1,diags={
#                                                                                                                   'NJN-P',
#                                                                                                                   'NJN-pi0f-pi0i',
#                                                                                                                 #   'N-pi0f-j-pi0i',
#                                                                                                                   'B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f',
#                                                                                                                   'N-pi0f-jPi','N-jPf-pi0i',
#                                                                                                                 #   'B2pt-j','W2pt-j','Z2pt-j',
#                                                                                                                 #   'N-pi0f-j-pi0i',
#                                                                                                                 #   'N-P-j',
#                                                                                                                 #   'T-jPf','T-pi0f-j'
#                                                                                                                   })
# run3pt(yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),'g5_j-',1,diags={
#                                                                                                                     'NJN-pi0f-pi0i',
#                                                                                                                 #   'NJN-P',
#                                                                                                                 #   'N-pi0f-j-pi0i',
#                                                                                                                 #   'B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f',
#                                                                                                                 #   'N-pi0f-jPi','N-jPf-pi0i',
#                                                                                                                 #   'B2pt-j','W2pt-j','Z2pt-j',
#                                                                                                                 #   'N-pi0f-j-pi0i',
#                                                                                                                 #   'N-P-j',
#                                                                                                                 #   'T-jPf','T-pi0f-j'
#                                                                                                                   })

# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'g5_j-',1,diags=yu.diags_all)
# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),'g5_j-',-1j,diags=yu.diags_all)
# run3pt(yu.getop('0,0,1','l2',('a','p')),yu.getop('0,0,1','l2',('N1pi0,a','p,pi0')),'g5_j-',-1j,diags=yu.diags_all)

In [ ]:
# updateEns(ensembles[1])
# diags={'NJN-pi0f-pi0i'}
# run3pt(yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),yu.getop('0,0,1','l2',('N1pi0,a','p,pi0')),'g5gx_j-',-1j,diags=diags)
# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l2',('a','p')),'g5gx_j-',-1j,diags={'NJN',})

# run3pt(yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),'g5gz_j-',-1j,diags=diags)
# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'g5gz_j-',-1j,diags={'NJN',})

In [ ]:
# updateEns(ensembles[1])
# diags={'NJN-P'}
# run3pt(yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),yu.getop('0,0,1','l2',('N1pi0,a','p,pi0')),'g5gx_j-',-1j,diags=diags)
# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l2',('a','p')),'g5gx_j-',-1j,diags={'NJN',})

# run3pt(yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),yu.getop('0,0,1','l1',('N1pi0,a','p,pi0')),'g5gz_j-',-1j,diags=diags)
# run3pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'g5gz_j-',-1j,diags={'NJN',})

In [ ]:
tfs=[10,12,14]

def run3pt2pt(opa,opb,insert,prefactor,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    
    needsVEV=yu.needsVEV(opa,opb,insert) and 'N-j' in diags and insert in ['id_j+','g5_j-'] 
    # print(needsVEV)
    
    dat2pt=[get2pt(opa,opa,yu.diags_all),get2pt(opb,opb,yu.diags_all)]
    dat3pt=[get3pt(opa,opb,insert+'_'+str(tf),diags)[:,1:tf] for tf in tfs]
    datVEVj=[data['VEV']['j'][insert]]
    dat=[dat3pt,dat2pt,datVEVj]

    def func(dat):
        t=yu.meanDeep(dat)
        
        t=[(t[0][i] - (t[1][0][tf]*t[2][0] if needsVEV else 0)  )/np.sqrt(
            np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
            np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
            np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
        )
           for i,tf in enumerate(tfs)]
           
        t=yu.prefactorDeep(t,prefactor)
        return t
    
    func(dat)

    (mean,err,cov)=yu.jackknife(dat,func)

    irow=0; icol=0
    for i_tf,tf in enumerate(tfs):
        tMean=mean[i_tf];tErr=err[i_tf]
        # print(tMean)
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
    axs[irow,icol].set_xticks(range(-6,7,1))
    axs[irow,icol].set_ylim(ylim1)
    
# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'id_j+',1,diags={'NJN'})

# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'gt_j+',1,diags={'NJN'})
# run3pt2pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'gt_j+',1,diags={'NJN'})

# run3pt2pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('N0pi1,a','n,pi+')),'gt_j+',1,diags=yu.diags_all)

# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,0','l1',('a','p')),'id_j+',1,diags={'N-j'})

# run3pt2pt(yu.getop('0,0,1','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'g5_j-',1,diags={'N-j'})

# updateEns(ensembles[1])
# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'id_j+',-1*0.05,diags=yu.diags_all)
# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'g5_j-',-1j,diags=yu.diags_all)

# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'gz_j+',-1j,diags=yu.diags_all)
# run3pt2pt(yu.getop('0,0,0','l1',('a','p')),yu.getop('0,0,1','l1',('a','p')),'g5gz_j-',-1j,diags=yu.diags_all)

# updateEns(ensembles[1])
# opa=yu.getop('0,0,0','l1',('a','p')); opb=yu.getop('0,0,1','l1',('a','p')); insert='gz_j-'; prefactor=lat.ZV[0]*-1j
# run3pt2pt(opa,opb,insert,prefactor,diags={'NJN'})
# run3pt2pt(opa,opb,insert,prefactor,diags={'N-j'})

In [ ]:
tfs=[10,12,14]

proj2mat={
    'P0':np.array([[1,0],[0,1]])/2,
    'Px':np.array([[0,1],[1,0]])/2,
    'Py':np.array([[0,-1j],[1j,0]])/2,
    'Pz':np.array([[1,0],[0,-1]])/2,
}

def run3pt2pt(ptofa,ptofb,pips,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    # fig.suptitle(title)
    
    # prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    
    # needsVEV=yu.needsVEV(opa,opb,insert) and 'N-j' in diags and insert in ['id_j+','g5_j-'] 
    # print(needsVEV)
    pta,ofa=ptofa; ptb,ofb=ptofb
    opa=yu.getop(pta,'l1',ofa); opb=yu.getop(ptb,'l1',ofb);
    
    AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
    def t_get3pt(pip,tf,diags):
        prefactor,insert,proj=pip
        proj=proj2mat[proj]
        proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
        t3pt=np.array([[get3pt(yu.getop(pta,l,ofa),yu.getop(ptb,m,ofb),insert+'_'+str(tf),diags)[:,1:tf] for m in ['l1','l2']] for l in ['l1','l2']])
        return prefactor*np.einsum('lmct,lm->ct',t3pt,proj2)
        
    dat2pt=[get2pt(opa,opa,yu.diags_all),get2pt(opb,opb,yu.diags_all)]
    dat3pt=[np.sum([t_get3pt(pip,tf,diags) for pip in pips], axis=0) for tf in tfs]
    # datVEVj=[data['VEV']['j'][insert]]
    dat=[dat3pt,dat2pt]

    def func(dat):
        t=yu.meanDeep(dat)
        
        t=[(t[0][i] )/np.sqrt(
            np.real(t[1][0][tf])*np.real(t[1][1][tf])*\
            np.real(t[1][0][1:tf][::-1])/np.real(t[1][0][1:tf])*\
            np.real(t[1][1][1:tf])/np.real(t[1][1][1:tf][::-1])
        )
           for i,tf in enumerate(tfs)]
           
        t=yu.prefactorDeep(t,1)
        return t

    (mean,err,cov)=yu.jackknife(dat,func)

    irow=0; icol=0
    for i_tf,tf in enumerate(tfs):
        tMean=mean[i_tf];tErr=err[i_tf]
        # print(tMean)
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
    axs[irow,icol].set_xticks(range(-6,7,1))
    axs[irow,icol].set_ylim(ylim1)

updateEns(ensembles[1])

mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
t_Fpi=92.9/lat.aInv
t_aQ2=(qk**2-(EN-mN)**2)
t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P

t_t=1/(np.sqrt(2*EN*(EN+mN)))

j='j-'

# ptofa=('0,0,0',('a','p')); ptofb=('0,0,0',('a','p')); 
# pips=[(lat.ZV[0],'gt_'+j,'P0')] 

# ptofa=('0,0,0',('a','p')); ptofb=('0,0,1',('a','p')); 
# pips=[(lat.ZV[0]*(np.sqrt(2*EN*(EN+mN)))/(EN+mN),'gt_'+j,'P0')]

# ptofa=('0,0,0',('a','p')); ptofb=('0,0,1',('a','p')); 
# pips=[(lat.ZV[0]*(np.sqrt(2*EN*(EN+mN)))/(-1j*qk),'gz_'+j,'P0')]

# ptofa=('0,0,0',('a','p')); ptofb=('0,0,1',('a','p')); 
# pips=[(lat.ZV[0]*(np.sqrt(2*EN*(EN+mN)))/(qk),'gy_'+j,'Px')]

# ptofa=('0,0,0',('a','p')); ptofb=('0,0,1',('a','p')); 
# pips=[(lat.ZV[0]*(EN-mN),'gt_j-','P0'),(lat.ZV[0]*-1j*(qk),'gz_'+j,'P0')]

# run3pt2pt(ptofa,ptofb,pips,diags={'NJN'})
# run3pt2pt(ptofa,ptofb,pips,diags={'N-j'})


# ptofa=('0,0,0',('a','p')); ptofb=('0,0,1',('a','p')); coeff=0
# pips=[(lat.ZV[0],'gt_'+j,'P0'),(lat.ZV[0]*(EN-mN)*coeff,'gt_'+j,'P0'),(lat.ZV[0]*-1j*(qk)*coeff,'gz_'+j,'P0')]
# run3pt2pt(ptofa,ptofb,pips,diags={'NJN'})
# ptofa=('0,0,0',('a','p')); ptofb=('0,0,1',('a','p')); coeff=-1*(EN-mN)/t_aQ2
# pips=[(lat.ZV[0],'gt_'+j,'P0'),(lat.ZV[0]*(EN-mN)*coeff,'gt_'+j,'P0'),(lat.ZV[0]*-1j*(qk)*coeff,'gz_'+j,'P0')]
# run3pt2pt(ptofa,ptofb,pips,diags={'NJN'})

updateEns(ensembles[0])

ptofa=('0,0,0',('a','p')); ptofb=('0,0,0',('a','p')); 
pips=[(1,'id_j+','P0')]
run3pt2pt(ptofa,ptofb,pips,diags={'NJN'},ylim1=[7,17])

ptofa=('0,0,1',('a','p')); ptofb=('0,0,1',('a','p')); 
pips=[(1,'id_j+','P0')]
run3pt2pt(ptofa,ptofb,pips,diags={'NJN'},ylim1=[7,17])

In [ ]:
tfs=[10,12,14]; tRef=1
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,0,:]/t[:,None]
    # return t[10]
    return np.mean(t[10:14+1,:],axis=0)

def run3pt2ptGEVP(ptofas,ptofbs,insert,prefactor,proj,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    pta,ofas=ptofas; ptb,ofbs=ptofbs
    needsVEV= pta==ptb and 'N-j' in diags and insert in ['id_j+','g5_j-'] and proj in ['P0','Pz']
    
    proj2mat={
        'P0':np.array([[1,0],[0,1]])/2,
        'Px':np.array([[0,1],[1,0]])/2,
        'Py':np.array([[0,-1j],[1j,0]])/2,
        'Pz':np.array([[1,0],[0,-1]])/2,
    }

    if proj in proj2mat.keys():
        proj=proj2mat[proj]
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
            
        Nparas=[yu.getNpar(op) for op in yu.getops(pta,'l1',ofas)]
        Nparbs=[yu.getNpar(op) for op in yu.getops(ptb,'l1',ofbs)]
        maska=np.array([[0 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        maskb=np.array([[0 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
        # maska=np.array([[1 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        # maskb=np.array([[1 if l==r==2 else 1 for l in Nparbs] for r in Nparbs]) 
        
        def t_get(tf):
            t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),insert+'_'+str(tf),diags)[:,1:tf]\
                for lb in ['l1','l2']] for la in ['l1','l2']]
            return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
        
        dat3pt=[t_get(tf) for tf in tfs]
        dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),yu.diags_all),
                get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),yu.diags_all),]
        datVEVj=[data['VEV']['j'][insert]]
        dat=[dat3pt,dat2pt,datVEVj]
        
        AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

        def func(dat):
            t=yu.meanDeep(dat)
            t3pt,t2pt,tVEVj=t
            
            (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
            (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
            (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
            (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
            
            eVec_al1=get_eVec(eVecs_al1)
            eVec_al2=get_eVec(eVecs_al2)
            eVec_bl1=get_eVec(eVecs_bl1)
            eVec_bl2=get_eVec(eVecs_bl2)
            
            t2pt_a=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
            t2pt_b=(np.einsum('i,tij,j,ij->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1,maskb)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2,maskb))/2
            
            if needsVEV:
                t2pt_sub=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)*proj[0,0]+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska)*proj[1,1])
            
            eVec_a=np.array([eVec_al1,eVec_al2])
            eVec_b=np.array([eVec_bl1,eVec_bl2])
            
            t=[(
                np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                -(t2pt_sub[tf]*tVEVj[0] if needsVEV else 0)
            )/np.sqrt(
                np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
            )
            for i,tf in enumerate(tfs)]
        
            t=yu.prefactorDeep(t,prefactor)
            return t
        
        (mean,err,cov)=yu.jackknife(dat,func)

        irow=0; icol=0
        for i_tf,tf in enumerate(tfs):
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
            # axs[irow,icol].errorbar(np.arange(1,tf),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)
        
updateEns(ensembles[0])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_jLoopless)
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_all)

# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'gt_j+',1,'P0',diags=yu.diags_jLoopless,ylim1=[4,5])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'gt_j+',1,'P0',diags=yu.diags_all,ylim1=[4,5])

# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'gt_j-',1,'P0',diags=yu.diags_jLoopless,ylim1=[1.3,1.5])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'gt_j-',1,'P0',diags=yu.diags_all,ylim1=[0.5,2.5])

# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-4.5,-2.5])
# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_all,ylim1=[-10,-4])
 
# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_all,ylim1=[-0.2,0.5])

# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5_j-',1,'Pz',diags=yu.diags_all)

# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.5,1.3])
# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_all,ylim1=[-0.5,1.3])

# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless)

In [ ]:
updateEns(ensembles[1])
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N0pi1,a','12')]),'gt_j-',1,'P0',diags=yu.diags_jLoopless,ylim1=[0.5,1.5])
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N0pi1,a','12')]),'gt_j-',1,'P0',diags=yu.diags_all,ylim1=[0.5,1.5])

run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N0pi1,a','12'),('N1pi0,a','12')]),'gz_j-',1j,'P0',diags=yu.diags_jLoopless,ylim1=[0,0.3])
run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N0pi1,a','12'),('N1pi0,a','12')]),'gz_j-',1j,'P0',diags=yu.diags_all,ylim1=[0,0.3])

In [ ]:
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gy_j-',-1j,'Py',diags=yu.diags_jLoopless,ylim1=[-1.35,-1.15])

In [ ]:
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_jLoopless)
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_all)

In [ ]:
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_all,ylim1=[-0.2,0.5])

# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-10,-2.5])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_all,ylim1=[-10,-2.5])

# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gz_j-',-1j,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.8,-0.2])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gz_j-',-1j,'Pz',diags=yu.diags_all,ylim1=[-0.8,-0.2])

In [ ]:
# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[0.44,0.86])
# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_all,ylim1=[0.44,0.86])

# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12')]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-2,2])
# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12')]),'g5_j-',1,'Pz',diags=yu.diags_all,ylim1=[-2,2])

# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[0.44,0.86])
# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_all,ylim1=[0.44,0.86])

# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-2,2])
# run3pt2ptGEVP(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5_j-',1,'Pz',diags=yu.diags_all,ylim1=[-2,2])

In [ ]:
# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
        
# run3pt2ptGEVP(('0,0,0',[('a','p')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-4.5,-2.5])
# run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-4.5,-2.5])

In [ ]:
tfs=[10,12,14]
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,0,:]/t[:,None]
    # return t[10]
    return np.mean(t[10:14+1,:],axis=0)

def run3pt2ptGEVP_compare_t0(ptofas,ptofbs,insert,prefactor,proj,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    pta,ofas=ptofas; ptb,ofbs=ptofbs
    needsVEV= pta==ptb and 'N-j' in diags and insert in ['id_j+','g5_j-'] and proj in ['P0','Pz']
    
    proj=np.array([[1,0],[0,1]])/2 if proj=='P0' else np.array([[1,0],[0,-1]])/2 if proj=='Pz' else proj
    
    for case in ['noGEVP',1,2,4,6]:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
        tRef=case if case!='noGEVP' else 1
            
        Nparas=[yu.getNpar(op) for op in yu.getops(pta,'l1',ofas)]
        Nparbs=[yu.getNpar(op) for op in yu.getops(ptb,'l1',ofbs)]
        maska=np.array([[0 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        maskb=np.array([[0 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
        # maska=np.array([[1 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        # maskb=np.array([[1 if l==r==2 else 1 for l in Nparbs] for r in Nparbs]) 
        
        def t_get(tf):
            t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),insert+'_'+str(tf),diags)[:,1:tf]\
                for lb in ['l1','l2']] for la in ['l1','l2']]
            return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
        
        dat3pt=[t_get(tf) for tf in tfs]
        dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),yu.diags_all),
                get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),yu.diags_all),]
        datVEVj=[data['VEV']['j'][insert]]
        dat=[dat3pt,dat2pt,datVEVj]
        
        AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

        def func(dat):
            t=yu.meanDeep(dat)
            t3pt,t2pt,tVEVj=t
            
            (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
            (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
            (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
            (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
            
            eVec_al1=get_eVec(eVecs_al1)
            eVec_al2=get_eVec(eVecs_al2)
            eVec_bl1=get_eVec(eVecs_bl1)
            eVec_bl2=get_eVec(eVecs_bl2)
            
            t2pt_a=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
            t2pt_b=(np.einsum('i,tij,j,ij->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1,maskb)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2,maskb))/2
            
            if needsVEV:
                t2pt_sub=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)*proj[0,0]+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska)*proj[1,1])
            
            eVec_a=np.array([eVec_al1,eVec_al2])
            eVec_b=np.array([eVec_bl1,eVec_bl2])
            
            t=[(
                np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                -(t2pt_sub[tf]*tVEVj[0] if needsVEV else 0)
            )/np.sqrt(
                np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
            )
            for i,tf in enumerate(tfs)]
        
            t=yu.prefactorDeep(t,prefactor)
            return t
        
        (mean,err,cov)=yu.jackknife(dat,func)
        
        colorDic={'noGEVP':'black',1:'r',2:'g',4:'b',6:'cyan'}

        irow=0; icol=0
        for i_tf,tf in enumerate(tfs):
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color=colorDic[case],fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
            # axs[irow,icol].errorbar(np.arange(1,tf),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)

In [ ]:
# run3pt2ptGEVP_compare_t0(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_jLoopless)
# run3pt2ptGEVP_compare_t0(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_all)

In [ ]:
# run3pt2ptGEVP_compare_t0(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# run3pt2ptGEVP_compare_t0(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_all,ylim1=[-0.2,0.5])

# run3pt2ptGEVP_compare_t0(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-10,-2.5])
# run3pt2ptGEVP_compare_t0(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_all,ylim1=[-10,-2.5])

# run3pt2ptGEVP_compare_t0(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gz_j-',-1j,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.8,-0.2])
# run3pt2ptGEVP_compare_t0(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gz_j-',-1j,'Pz',diags=yu.diags_all,ylim1=[-0.8,-0.2])

In [ ]:
# run3pt2ptGEVP_compare_t0(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[0.44,0.86])
# run3pt2ptGEVP_compare_t0(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5gt_j-',1,'Pz',diags=yu.diags_all,ylim1=[0.44,0.86])

# run3pt2ptGEVP_compare_t0(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-2,2])
# run3pt2ptGEVP_compare_t0(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),'g5_j-',1,'Pz',diags=yu.diags_all,ylim1=[-2,2])

In [ ]:
tfs=[10,12,14]; tRef=1
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,0,:]/t[:,None]
    # return t[10]
    return np.mean(t[10:14+1,:],axis=0)

def runFF(ptofas,ptofbs,FF,diags=yu.diags_all,title=None,ylim1=None):
    
    mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
    t_5=-(2*mN)/(K*qk)
    t_A=1j*(2*mN)/(K*(EN+mN))
    t_Pt=(4*mN**2)/(K*qk*(mN-EN)); t_Pt2=(2*mN)/(mN-EN)
    t_Pi=-1j*(4*mN**2)/(K*qk**2); t_Pi2=-(2*mN*(EN+mN))/(qk**2)
    
    pips={
        'GS':[(1*lat.amu*lat.aInv,'id_j+','P0')],
        'G5':[(t_5*lat.ZS,'g5_j-','Pz')],
        'GA0':[(1j*lat.ZA,'g5gz_j-','Pz')],
        'GA':[(t_A*(1/2)*lat.ZA,'g5gx_j-','Px'),(t_A*(1/2)*lat.ZA,'g5gy_j-','Py')],
        'GPt':[(t_Pt2*t_A*(-1/2)*lat.ZA,'g5gx_j-','Px'),(t_Pt2*t_A*(-1/2)*lat.ZA,'g5gy_j-','Py'),(t_Pt*lat.ZA,'g5gt_j-','Pz')],
        'GPi':[(t_Pi2*t_A*(-1/2)*lat.ZA,'g5gx_j-','Px'),(t_Pi2*t_A*(-1/2)*lat.ZA,'g5gy_j-','Py'),(t_Pi*lat.ZA,'g5gz_j-','Pz')],
    }[FF]
    
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    pta,ofas=ptofas; ptb,ofbs=ptofbs
    needsVEV= pta==ptb and 'N-j' in diags and FF in ['GS'] 
    
    proj2mat={
        'P0':np.array([[1,0],[0,1]])/2,
        'Px':np.array([[0,1],[1,0]])/2,
        'Py':np.array([[0,-1j],[1j,0]])/2,
        'Pz':np.array([[1,0],[0,-1]])/2,
    }
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
            
        Nparas=[yu.getNpar(op) for op in yu.getops(pta,'l1',ofas)]
        Nparbs=[yu.getNpar(op) for op in yu.getops(ptb,'l1',ofbs)]
        maska=np.array([[0 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        maskb=np.array([[0 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
        # maska=np.array([[1 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        # maskb=np.array([[1 if l==r==2 else 1 for l in Nparbs] for r in Nparbs]) 
        
        def t_get(tf,t_insert):
            t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                for lb in ['l1','l2']] for la in ['l1','l2']]
            return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
        
        dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
        dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),yu.diags_all),
                get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),yu.diags_all),]
        datVEVj=[data['VEV']['j']['id_j+']]
        dat=[dat3pt,dat2pt,datVEVj]
        
        AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

        def func(dat):
            t=yu.meanDeep(dat)
            t3pt,t2pt,tVEVj=t
            
            (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
            (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
            (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
            (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
            
            eVec_al1=get_eVec(eVecs_al1)
            eVec_al2=get_eVec(eVecs_al2)
            eVec_bl1=get_eVec(eVecs_bl1)
            eVec_bl2=get_eVec(eVecs_bl2)
            
            t2pt_a=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
            t2pt_b=(np.einsum('i,tij,j,ij->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1,maskb)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2,maskb))/2
            
            if needsVEV:
                t2pt_sub=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
            
            eVec_a=np.array([eVec_al1,eVec_al2])
            eVec_b=np.array([eVec_bl1,eVec_bl2])
            
            def t_get2(i):
                res=0
                for i_pip,pip in enumerate(pips):
                    prefactor,insert,proj=pip
                    proj=proj2mat[proj]
                    res+=prefactor*np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i_pip*len(tfs)+i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                return res
            
            t=[(
                t_get2(i)
                -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
            )/np.sqrt(
                np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
            )
            for i,tf in enumerate(tfs)]
        
            t=yu.prefactorDeep(t,1)
            return t
        
        (mean,err,cov)=yu.jackknife(dat,func)

        irow=0; icol=0
        for i_tf,tf in enumerate(tfs):
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
            # axs[irow,icol].errorbar(np.arange(1,tf),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)
    axs[0,0].axis('auto')